### 1. Data 로드

In [ ]:
import importlib
import battery_data_processor
# 모듈 재로딩 (수정 후 필수!)
importlib.reload(battery_data_processor)

# 분석할 경로 리스트 (사용자가 수정)
paths = [
    r"C:\Users\Ryu\Python_project\data\dataprocess_2601\Rawdata\A1_MP1_4500mAh_T23_1",
    # 추가 경로를 여기에 입력하세요
]

print(f"분석 대상 경로 개수: {len(paths)}")

# 데이터 로드
data = battery_data_processor.process_and_combine(paths)
data = battery_data_processor.process_all_channels(data)
data = battery_data_processor.categorize_all_channels(data)

### 1.1 데이터 구조 확인

In [ ]:
# 첫 번째 채널의 데이터 구조 확인
channel_keys = list(data['channels'].keys())
first_channel_key = channel_keys[0]
channel_data = data['channels'][first_channel_key]

print(f"선택된 채널: {first_channel_key}")
print(f"Cycler Type: {channel_data['cycler_type']}")
print(f"Capacity: {channel_data['capacity_mAh']} mAh")
print(f"\n=== 데이터 구조 ===")
print(f"cycle 키 존재: {'cycle' in channel_data}")
print(f"cycle_summary 키 존재: {'cycle_summary' in channel_data}")
print(f"cycle_steps 키 존재: {'cycle_steps' in channel_data}")

if channel_data.get('cycle_summary') is not None:
    summary = channel_data['cycle_summary']
    print(f"\n=== 사이클 대표 용량 (Condition == 8) ===")
    print(f"  행 수: {len(summary):,}")
    print(f"  Condition 값: {summary['Condition'].unique()}")
    print(f"  Cycle 범위: {summary['Cycle'].min()} ~ {summary['Cycle'].max()}")
    print(f"  첫 5개 행:")
    print(summary[['Cycle', 'Condition', 'ChgCap_mAh', 'DchgCap_mAh']].head())

if channel_data.get('cycle_steps') is not None:
    steps = channel_data['cycle_steps']
    print(f"\n=== 스텝별 용량 (Condition != 8) ===")
    print(f"  행 수: {len(steps):,}")
    print(f"  Condition 값: {sorted(steps['Condition'].unique())}")
    print(f"  Condition 분포:")
    print(steps['Condition'].value_counts().sort_index())

### 2. 데이터 시각화

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# 한글 폰트 설정 (선택사항)
try:
    plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
    plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지
except:
    print("한글 폰트 설정 실패, 기본 폰트 사용")

#### 2.1 Cycle 데이터 시각화 (사이클 대표 용량)

In [ ]:
# cycle_summary 사용 (Condition == 8, 사이클 대표 용량)
if channel_data.get('cycle_summary') is not None:
    df_cycle = channel_data['cycle_summary']
    
    # 그림 크기 설정
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(f'Cycle 데이터 분석: {first_channel_key} (Condition==8)', fontsize=16, fontweight='bold')
    
    # 1. Discharge Capacity vs Cycle
    ax1 = axes[0, 0]
    if 'DchgCap_mAh' in df_cycle.columns:
        ax1.plot(df_cycle['Cycle'], df_cycle['DchgCap_mAh'], 'o-', markersize=3, linewidth=1, color='red')
        ax1.set_xlabel('Cycle Number')
        ax1.set_ylabel('Discharge Capacity (mAh)')
        ax1.set_title('방전 용량 변화')
        ax1.grid(True, alpha=0.3)
    
    # 2. Charge/Discharge Capacity
    ax2 = axes[0, 1]
    if 'DchgCap_mAh' in df_cycle.columns and 'ChgCap_mAh' in df_cycle.columns:
        ax2.plot(df_cycle['Cycle'], df_cycle['ChgCap_mAh'], 'b-', label='Charge', linewidth=1.5, alpha=0.7)
        ax2.plot(df_cycle['Cycle'], df_cycle['DchgCap_mAh'], 'r-', label='Discharge', linewidth=1.5, alpha=0.7)
        ax2.set_xlabel('Cycle Number')
        ax2.set_ylabel('Capacity (mAh)')
        ax2.set_title('충방전 용량 비교')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
    
    # 3. Temperature vs Cycle
    ax3 = axes[1, 0]
    if 'Temp_C' in df_cycle.columns:
        ax3.plot(df_cycle['Cycle'], df_cycle['Temp_C'], 'g-', linewidth=1)
        ax3.set_xlabel('Cycle Number')
        ax3.set_ylabel('Temperature (°C)')
        ax3.set_title('온도 변화')
        ax3.grid(True, alpha=0.3)
    
    # 4. Coulombic Efficiency
    ax4 = axes[1, 1]
    if 'DchgCap_mAh' in df_cycle.columns and 'ChgCap_mAh' in df_cycle.columns:
        # 0으로 나누기 방지
        efficiency = np.where(df_cycle['ChgCap_mAh'] > 0, 
                            df_cycle['DchgCap_mAh'] / df_cycle['ChgCap_mAh'] * 100, 
                            np.nan)
        ax4.plot(df_cycle['Cycle'], efficiency, 'purple', linewidth=1)
        ax4.set_xlabel('Cycle Number')
        ax4.set_ylabel('Coulombic Efficiency (%)')
        ax4.set_title('쿨롱 효율')
        ax4.set_ylim([90, 102])
        ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 통계 정보 출력
    print("\n=== Cycle 데이터 통계 (사이클 대표 용량) ===")
    print(f"총 사이클 수: {len(df_cycle)}")
    if 'DchgCap_mAh' in df_cycle.columns:
        print(f"방전 용량 범위: {df_cycle['DchgCap_mAh'].min():.2f} ~ {df_cycle['DchgCap_mAh'].max():.2f} mAh")
        print(f"방전 용량 평균: {df_cycle['DchgCap_mAh'].mean():.2f} mAh")
    if 'Temp_C' in df_cycle.columns:
        print(f"온도 범위: {df_cycle['Temp_C'].min():.2f} ~ {df_cycle['Temp_C'].max():.2f} °C")
else:
    print("Cycle summary 데이터가 없습니다.")

#### 2.2 스텝별 용량 분석 (Condition != 8)

In [ ]:
# 스텝별 용량 확인 (Condition != 8)
if channel_data.get('cycle_steps') is not None:
    df_steps = channel_data['cycle_steps']
    
    print(f"=== 스텝별 용량 분석 ===")
    print(f"총 행 수: {len(df_steps):,}")
    print(f"\nCondition별 분포:")
    print(df_steps['Condition'].value_counts().sort_index())
    
    # 특정 사이클 (예: Cycle 0)의 스텝별 용량 확인
    cycle_num = 0
    cycle_0_steps = df_steps[df_steps['Cycle'] == cycle_num]
    
    if len(cycle_0_steps) > 0:
        print(f"\n=== Cycle {cycle_num}의 스텝별 상세 ===")
        print(cycle_0_steps[['Cycle', 'Condition', 'ChgCap_mAh', 'DchgCap_mAh', 'EndState']].to_string())
        
        # 스텝별 용량 시각화
        fig, ax = plt.subplots(figsize=(12, 6))
        
        # Condition별로 다른 색상 사용
        conditions = sorted(cycle_0_steps['Condition'].unique())
        colors = plt.cm.tab10(np.linspace(0, 1, len(conditions)))
        
        for idx, cond in enumerate(conditions):
            cond_data = cycle_0_steps[cycle_0_steps['Condition'] == cond]
            ax.bar(idx, cond_data['DchgCap_mAh'].values[0] if len(cond_data) > 0 else 0, 
                  color=colors[idx], label=f'Condition {cond}', alpha=0.7)
        
        ax.set_xlabel('Condition')
        ax.set_ylabel('Discharge Capacity (mAh)')
        ax.set_title(f'Cycle {cycle_num}의 Condition별 방전 용량')
        ax.legend()
        ax.grid(True, alpha=0.3, axis='y')
        plt.tight_layout()
        plt.show()
    else:
        print(f"\nCycle {cycle_num}의 스텝 데이터가 없습니다.")
else:
    print("Cycle steps 데이터가 없습니다.")

#### 2.3 헬퍼 함수 사용 예시

In [ ]:
# 헬퍼 함수로 사이클 대표 용량 가져오기
summary = battery_data_processor.get_cycle_summary(data, 0)

if summary is not None:
    print("\n첫 5개 사이클의 방전 용량:")
    print(summary[['Cycle', 'DchgCap_mAh']].head())

print("\n" + "="*50)

# 헬퍼 함수로 스텝별 용량 가져오기
steps = battery_data_processor.get_cycle_steps(data, 0)

if steps is not None:
    print("\nCycle 0의 스텝별 용량:")
    print(steps[steps['Cycle'] == 0][['Cycle', 'Condition', 'DchgCap_mAh']])

#### 2.4 Profile 데이터 시각화 (특정 사이클)

In [ ]:
# Profile이 cycle_list로 처리된 경우
if isinstance(channel_data['profile'], list):
    cycle_list = channel_data['profile']
    
    # 시각화할 사이클 선택
    cycles_to_plot = [0, 10, 50, 100, 200, 300]
    cycles_to_plot = [c for c in cycles_to_plot if c < len(cycle_list)]
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(f'Profile 데이터 분석: {first_channel_key}', fontsize=16, fontweight='bold')
    
    # 1. Voltage vs Time
    ax1 = axes[0, 0]
    for cycle_idx in cycles_to_plot:
        cycle_df = cycle_list[cycle_idx]
        ax1.plot(cycle_df['time_cyc'] / 3600, cycle_df['Voltage_V'], label=f'Cycle {cycle_idx}', linewidth=1.5)
    ax1.set_xlabel('Time (hours)')
    ax1.set_ylabel('Voltage (V)')
    ax1.set_title('전압 프로파일')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Current vs Time
    ax2 = axes[0, 1]
    for cycle_idx in cycles_to_plot:
        cycle_df = cycle_list[cycle_idx]
        ax2.plot(cycle_df['time_cyc'] / 3600, cycle_df['Current_mA'], label=f'Cycle {cycle_idx}', linewidth=1.5)
    ax2.set_xlabel('Time (hours)')
    ax2.set_ylabel('Current (mA)')
    ax2.set_title('전류 프로파일')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Voltage vs Capacity
    ax3 = axes[1, 0]
    for cycle_idx in cycles_to_plot:
        cycle_df = cycle_list[cycle_idx]
        if 'Capa_cyc' in cycle_df.columns:
            ax3.plot(cycle_df['Capa_cyc'].abs(), cycle_df['Voltage_V'], label=f'Cycle {cycle_idx}', linewidth=1.5)
    ax3.set_xlabel('Capacity (mAh)')
    ax3.set_ylabel('Voltage (V)')
    ax3.set_title('전압-용량 곡선')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Temperature vs Time
    ax4 = axes[1, 1]
    for cycle_idx in cycles_to_plot:
        cycle_df = cycle_list[cycle_idx]
        if 'Temp_C' in cycle_df.columns:
            ax4.plot(cycle_df['time_cyc'] / 3600, cycle_df['Temp_C'], label=f'Cycle {cycle_idx}', linewidth=1.5)
    ax4.set_xlabel('Time (hours)')
    ax4.set_ylabel('Temperature (°C)')
    ax4.set_title('온도 프로파일')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n총 사이클 수: {len(cycle_list)}개")
    print(f"시각화된 사이클: {cycles_to_plot}")
else:
    print("Profile 데이터가 처리되지 않았습니다. process_all_channels()를 먼저 실행하세요.")

#### 2.5 카테고리별 사이클 분석

In [ ]:
# 카테고리별 사이클 분석
if 'cycle_list' in channel_data:
    categories = channel_data['cycle_list']
    cycle_list = channel_data['profile']
    
    # 카테고리별 통계
    print("=== 카테고리별 사이클 통계 ===")
    for category, indices in categories.items():
        if indices:
            print(f"\n[{category}]: {len(indices)}개")
            print(f"  사이클 인덱스: {indices[:10]}{'...' if len(indices) > 10 else ''}")
    
    # 카테고리별 대표 사이클 비교
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(f'카테고리별 대표 사이클 비교: {first_channel_key}', fontsize=16, fontweight='bold')
    
    colors = {'RPT': 'blue', 'SOC_Definition': 'green', 
              'Resistance_Measurement': 'red', 'Accelerated_Aging': 'orange', 'Unknown': 'gray'}
    
    ax1 = axes[0, 0]
    ax2 = axes[0, 1]
    ax3 = axes[1, 0]
    ax4 = axes[1, 1]
    
    for category, indices in categories.items():
        if indices:
            cycle_df = cycle_list[indices[0]]
            color = colors.get(category, 'black')
            
            ax1.plot(cycle_df['time_cyc'] / 3600, cycle_df['Voltage_V'], 
                    label=category, color=color, linewidth=1.5, alpha=0.7)
            
            ax2.plot(cycle_df['time_cyc'] / 3600, cycle_df['Current_mA'], 
                    label=category, color=color, linewidth=1.5, alpha=0.7)
            
            if 'Capa_cyc' in cycle_df.columns:
                ax3.plot(cycle_df['Capa_cyc'].abs(), cycle_df['Voltage_V'], 
                        label=category, color=color, linewidth=1.5, alpha=0.7)
            
            if 'Crate' in cycle_df.columns:
                ax4.hist(cycle_df['Crate'].abs(), bins=50, alpha=0.5, 
                        label=category, color=color, edgecolor='black')
    
    ax1.set_xlabel('Time (hours)')
    ax1.set_ylabel('Voltage (V)')
    ax1.set_title('전압 프로파일 비교')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    ax2.set_xlabel('Time (hours)')
    ax2.set_ylabel('Current (mA)')
    ax2.set_title('전류 프로파일 비교')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    ax3.set_xlabel('Capacity (mAh)')
    ax3.set_ylabel('Voltage (V)')
    ax3.set_title('전압-용량 곡선 비교')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    ax4.set_xlabel('C-rate')
    ax4.set_ylabel('Frequency')
    ax4.set_title('C-rate 분포')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("카테고리 정보가 없습니다. categorize_all_channels()를 먼저 실행하세요.")

#### 2.6 전체 채널 비교 (사이클 대표 용량)

In [ ]:
# 모든 채널의 Discharge Capacity 비교 (cycle_summary 사용)
fig, ax = plt.subplots(figsize=(15, 6))

for channel_key, channel_data in data['channels'].items():
    # cycle_summary 사용 (Condition == 8만)
    if channel_data.get('cycle_summary') is not None:
        df_summary = channel_data['cycle_summary']
        
        if 'DchgCap_mAh' in df_summary.columns:
            ax.plot(df_summary['Cycle'], df_summary['DchgCap_mAh'], 
                   'o-', label=channel_key, markersize=2, linewidth=1, alpha=0.8)

ax.set_xlabel('Cycle Number', fontsize=12)
ax.set_ylabel('Discharge Capacity (mAh)', fontsize=12)
ax.set_title('전체 채널 방전 용량 비교 (사이클 대표 용량)', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n총 {len(data['channels'])}개 채널 비교 완료")